In [ ]:
!pip install pydantic

## Processing data using dicts

Dictionaries are the backbone of python data structures, but it is very easy to miss errors with them because they do not enforce what kind of data you put into them.

In [ ]:
def report_pet(pet_dict):
    print(f"My name is {pet_dict['name']} and I need {pet_dict['n_legs'] / 2} pairs of pants")

In [ ]:
json_1 = {"name": "Mittens", "n_legs": 4}
json_2 = {"name": "Slither", "n_legs": 0}
json_3 = {"name": "Skitter", "n_legs": "8"}
json_4 = {"n_legs": 6}

In [ ]:
report_pet(json_1)

In [ ]:
report_pet(json_2)

The first two pets work fine because their dictionaries have data that happens to be valid. But things start to go wrong if we pass the wrong data type

In [ ]:
report_pet(json_3)

This error only comes up when we run our function to report on the pet - it doesn't check the data any earlier.

In [ ]:
report_pet(json_4)

And when our dictionary is missing an entire field, we need to figure out what the "key error" is.

## Processing data with Pydantic

[Pydantic](https://docs.pydantic.dev/latest/https://docs.pydantic.dev/latest/) uses python type hints to define a class - a way of stating the exact shape of data we expect to receive.

In [ ]:
from pydantic import BaseModel

class PydanticPet(BaseModel):
    name: str
    n_legs: int

def report_pypet(pypet: PydanticPet):
    print(f"My name is {pypet.name} and I need {pypet.n_legs / 2} pairs of pants")

Note that we aren't accessing dictionary keys with `["strings"]` that may or may not succeed, but instead using dot notation `pypet.name` because we _know_ that every `PydanticPet` instance has an attribute called `name`.

In [ ]:
pypet_1 = PydanticPet(**json_1)
# Using ** is a python trick that passes a dictionary to a function by "expanding" it and putting in the key names as arugments
# pypet_1 = PydanticPet(name="Mittens", n_legs=4)

In [ ]:
pypet_1

In [ ]:
report_pypet(pypet_1)

In [ ]:
pypet_2 = PydanticPet(**json_2)

In [ ]:
pypet_2

In [ ]:
report_pypet(pypet_2)

Pydantic can automate certain kinds of data parsing, such as converting the string `"8"` to the integer `8`.

In [ ]:
pypet_3 = PydanticPet(**json_3)

In [ ]:
pypet_3

In [ ]:
report_pypet(pypet_3)

In [ ]:
PydanticPet(**json_4)

Pydantic raises a `ValidationError` that provides a clear reason why the data passed in was invalid.

## Nesting and lists

Pydantic models can refer to other pydantic models, and can nest lists of data too.

In [ ]:
class PetDaycare(BaseModel):
    name: str
    founding_year: int | None # This indicates that founding_year is an optional attribute
    current_pets: list[PydanticPet] = []

In [ ]:
local_daycare = PetDaycare(name="All Things That Crawl")

In [ ]:
local_daycare

In [ ]:
local_daycare.current_pets.append(pypet_1)
local_daycare.current_pets.append(pypet_2)
local_daycare.current_pets.append(pypet_3)

In [ ]:
local_daycare

In [ ]:
for pet in local_daycare.current_pets:
    report_pypet(pet)

One of the biggest uses of pydantic is serializing data to JSON to be used in API servers.

In [ ]:
local_daycare.json()

Pydantic also can autogenerate a JSONSchema that can power API documentation pages.

In [ ]:
PetDaycare.schema()